In [ ]:
# ************************
# Import Packages
# ************************
import pandas as pd
import numpy as np
import re
from IPython.display import Image, display, Markdown, HTML
import copy
from pprint import pprint
import helpers
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# ************************
# User Defined Parameters
# ************************
infile = "../pyCIMS_model_description.xlsm"
NODE_COL = "Node"
TYPE_COL = "Demand?"
MODEL_SHEET = "Model"
EXTRA_COL = 'Demand?'

In [ ]:
# ************************
# Read in the data
# ************************

# Read model_description from excel
mxl = pd.read_excel(infile, sheet_name=None, header=1)

# Read the model sheet into a dataframe
model_df = mxl[MODEL_SHEET].replace({pd.np.nan: None})

# Adjust index to correspond to Excel line numbers
model_df.index += 3  #(+1: 0 vs 1 origin, +1: header skip, +1: column headers)

# Convert all column names to strings (years were ints)
model_df.columns = [str(c) for c in model_df.columns]

# Find the columns, separated by whether they are year columns or not
node_cols, year_cols = helpers.get_node_cols(model_df, extra_col=EXTRA_COL)
all_cols = np.concatenate((node_cols, year_cols))

# Create the model dataframe
mdf = model_df.loc[1:,all_cols] # drop irrelevant columns and skip first, empty row

In [ ]:
# # ************************
# # Extract Node Dataframes
# # ************************
# # determine, row ranges for each node def, based on non-empty Node field
# node_rows = mdf.Node[~mdf.Node.isnull()] # does not work if node names have been filled in
# node_rows.index.name = "Row Number"
# last_row = mdf.index[-1]
# node_start_ends = zip(node_rows.index,
#                       node_rows.index[1:].tolist() + [last_row])

# # extract Node DataFrames, at this point still including Technologies
# node_dfs = {}
# non_node_cols = mdf.columns != NODE_COL
# for s, e in node_start_ends:
#     node_name = mdf.Node[s]
#     node_df = mdf.loc[s+1:e-1]
#     node_df = node_df.loc[helpers.non_empty_rows(node_df), non_node_cols]
#     node_dfs[node_name] = node_df

In [ ]:
# ************************
# Extract Node Dataframes
# ************************
# determine, row ranges for each node def, based on non-empty Node field
node_rows = mdf.Node[~mdf.Node.isnull()] # does not work if node names have been filled in
node_rows.index.name = "Row Number"
last_row = mdf.index[-1]
node_start_ends = zip(node_rows.index,
                      node_rows.index[1:].tolist() + [last_row])

# extract Node DataFrames, at this point still including Technologies
node_dfs = {}
non_node_cols = mdf.columns != NODE_COL
for s, e in node_start_ends:
#     node_name = mdf.Node[s]
    node_df = mdf.loc[s+1:e-1]
    node_df = node_df.loc[helpers.non_empty_rows(node_df), non_node_cols]
    
    try:
        node_name = list(node_df[node_df['Parameter']=='Service provided']['Branch'])[0]
    except IndexError:
        continue   

    node_dfs[node_name] = node_df

In [ ]:
# ************************
# Extract Tech Dataframes
# ************************
# Extract tech dfs from node df's and rewrite node df without techs
tech_dfs = {}
for nn, ndf in node_dfs.items():
    if any(ndf.Parameter.isin(["Technology", "Service"])):  # Technologies can also be called Services
        tdfs = {}
        first_row, last_row = ndf.index[0], ndf.index[-1]
        tech_rows = ndf.loc[ndf.Parameter.isin(["Technology", "Service"])].index
        for trs, tre in zip(tech_rows, tech_rows[1:].tolist()+[last_row]):
            tech_df = ndf.loc[trs:tre-1]
            tech_name = tech_df.iloc[0].Value
            tdfs[tech_name] = tech_df
        tech_dfs[nn] = tdfs
        node_dfs[nn] = ndf.loc[:tech_rows[0]-1]

In [ ]:
# ************************
# Display Model Dataframes
# ************************
# display content of entire Model dataframe as separate df's
# for nn, ndf in node_dfs.items():
#     display(Markdown("Node: **{}**".format(nn)))
#     helpers.display_df(ndf)
#     if nn in tech_dfs:
#         for tech_name, tdf in tech_dfs[nn].items():
#             display(Markdown("Node / Technology: **{} / {}**".format(nn, tech_name)))
#             helpers.display_df(tdf)

## Jillian's Stuff

In [ ]:
# ************************
# Functions
# ************************
def add_node_data(g, current_node,):
    """
    @param g: NetworkX DiGraph. 
    @param current_node: The 
    @return: None. Modified the graph g instead. 
    Given a graph g and node current_node add current_node to g, along with all of its associated data."""

    # Copy the current node dataframe
    current_node_df = copy.deepcopy(node_dfs[current_node])

    # 1. we are going to create a node in the graph
    g.add_node(current_node)

    # 2. We will store the Supply/Demand Type of node. This is a special case. 
    typ = list(current_node_df[TYPE_COL])[0]
    g.node[current_node]['type'] = typ.lower() if typ else 'standard'
    # Drop Demand column
    current_node_df = current_node_df.drop(TYPE_COL, axis=1)

    # 3. We will store the Competition Type of the node at the node level. This is another special case. 
    comp_list = list(current_node_df[current_node_df['Parameter']=='Competition type']['Value'])
    if len(set(comp_list)) == 1: 
        comp_type = comp_list[0]
        g.node[current_node]['competition_type'] = comp_type.lower()
    elif len(set(comp_list)) > 1:
        print("TOO MANY COMPETITION TYPES")
    # Get rid of competition type row
    current_node_df = current_node_df[current_node_df['Parameter']!='Competition type']

    # 4. For the remaining rows, group data by year. 
    # Get Year Columns
    years = [c for c in current_node_df.columns if helpers.is_year(c)]

    # Get Non-Year Columns
    non_years = [c for c in current_node_df.columns if not helpers.is_year(c)]

    # For each year: 
    for y in years:
        year_df = current_node_df[non_years + [y]]
        year_dict = {}

        for parameter, source, branch, unit, value, year_value in zip(*[year_df[c] for c in year_df.columns]):
            if parameter in year_dict.keys():
                pass
            else:
                year_dict[parameter] = {}

            dct = {'source': source,
                   'branch': branch, 
                   'unit': unit, 
                   'year_value': year_value}
#             # Clean Dict
#             clean_dict = {k: v for k, v in dct.items() if v is not None}
            
            year_dict[parameter][value] = dct
            
        # Add data to node
        g.node[current_node][y] = year_dict
    
    
def add_tech_data(node, tech, tech_df):
    # TODO: I think we need to differentiate between Technologies and Services. 

    t_df = copy.deepcopy(tech_df)
    
    # 1. Find whether technology is a service or a technology. 
    #    Then remove the row that indicates this is a service or technology. 
    service_technology = 'service' if (t_df['Parameter']=='Service').any() else 'technology'
    t_df = t_df[~t_df['Parameter'].isin(['Service', 'Technology'])]

    # 2. Remove the Demand? column
    t_df = t_df.drop('Demand?', axis=1)

    # VERY SIMILAR to what we do for nodes. But not quite. Because we don't use the value column anymore
    # 4. For the remaining rows, group data by year. 
    # Get Year Columns
    years = [c for c in t_df.columns if helpers.is_year(c)]

    # Get Non-Year Columns
    non_years = [c for c in t_df.columns if not helpers.is_year(c)]

    # For each year: 
    for y in years:
        year_df = t_df[non_years + [y]]
        year_dict = {}

        for parameter, source, branch, unit, value, year_value in zip(*[year_df[c] for c in year_df.columns]):
            dct = {'source': source,
                       'branch': branch, 
                       'unit': unit, 
                       'year_value': year_value}
            
            if parameter in year_dict.keys():
                if type(year_dict[parameter]) is list:
                    year_dict[parameter] = year_dict[parameter].append(dct)
                else: 
                    year_dict[parameter] = [year_dict[parameter], dct] 
            else:
                year_dict[parameter] = dct

        # Add technologies key if needed
        if not 'technologies' in g.node[node][y].keys():
            g.node[node][y]['technologies'] = {}
            
        # Add the technology specific data for that year
        g.node[node][y]['technologies'][tech]= year_dict
        
def search_nodes(g, search_term):
    """Search nodes to see if there is one that contains the search term in the final component of its name"""
    def search(name):
        components = name.split('.')
        last_comp = components[-1]
        
        return search_term.lower() in last_comp.lower()
    
    return [n for n in g.nodes if search(n)]


In [ ]:
# ************************
# Add Node Data
# ************************

# Create Graph
g = nx.DiGraph()

# Add each node and its associated data to the Graph
for n in node_dfs.keys():
    add_node_data(g, n)
    print("************* {} *************".format(n))
    pprint(g.node[n])

In [ ]:
clothes_node = search_nodes(g, 'clothes')[0]
pprint(g.node[clothes_node]['2015'])

In [ ]:
for node in tech_dfs:
    # Add technologies key to node data
    node_techs_dict = {}
    for tech in tech_dfs[node]: 
        add_tech_data(node, tech, tech_dfs[node][tech])

In [ ]:
dw = search_nodes(g, 'dishwashing')[0]
pprint(g.nodes[dw])

# Data Structure Description
We have a NetworkX graph structure composed of nodes and edges. Nodes represent services within the model. Edges are directional and represent a request/provide relationship between two services. Edges flow from requesting nodes to providing nodes (down the tree). 

## Node Data
Each node has its own associated data, stored at the node in nested dictionaries. Each node's data dictionary contains 13 keys. 11 of these correspond to years within the simulation and have dictionaries as their values. The other two keys are: 
* `type`: One of supply, demand, or standard. This specifies whether a node is used during the supply or demand portion of the algorithm. Standard nodes are used in both (they root the tree and have children of both supply and demand types). Supply nodes are used only during supply calculations. Similarly, demand nodes are used only during demand calculations. 
* `competition_type`: One of 'tech_compete' or 'winner_takes_all'. Specifies the competition type that will be used to calculate supply/demand at that node. Not all nodes will have this attribute. 

### Year Dictionaries
The year dictionaries, contain all the data specific to a given year. These year specific dictionaries contain keys for regular attributes (see below for description) and may contain a `"technologies"` key. 
* regular attributes correspond to non-technology parameters from the model description. Examples include price, service requested, service provided, and heterogeniety. Regular attributes are given a key corresponding to the `Parameter` column from the model description and a value that is a dictionary composed from the `branch`, `source`, `unit` and `value` columns from the model description in addition to the and year-specific values provided in the 2000 ... 2050 columns within the model description. They follow the format below: 
```
'parameter1': {'value1': {'branch': <val>, 
                          'source': <val>, 
                          'unit': <val>, 
                          'year_value': <val>}}
```
See the output from the cell below to see an example of node data containing regular attributes. For simplicity I have only included the 2020 subdictionary. 

In [ ]:
alberta = search_nodes(g, 'alberta')[0]
pprint(g.node[alberta])

* `technologies`: Will only be present in nodes with a 'tech_compete' or 'winner_takes_all' competition type. Once again, the value will be a dictionary of dictionaries, with one dictionary for each technology. Each technologies entry is set up as shown below, with <val> corresponding to the value found in the appropriate year column in the model description. See the next cell's output for an example of a technology dictionary.  
```
'technology1': {'parameter1' : {'branch': <val>
                                'source': <val>, 
                                'unit': <val>, 
                                'year_value': <val>}, 
                'parameter2' : {'branch': <val>
                                'source': <val>, 
                                'unit': <val>, 
                                'year_value': <val>}}
```
    

In [ ]:
print("*****************************")
print("Electric Baseboard Technology")
print("*****************************")
space_heating = search_nodes(g, 'space heating')[0]

pprint(g.node[space_heating]['2000']['technologies']['Electric baseboard'])

# Add Edges

In [ ]:
def add_edges(node, df, g):
    # Find edges based on Requester/Provider relationships
    # ----------------------------------------------------
    # Find all nodes node is requesting services from
    providers = df[df['Parameter']=='Service requested']['Branch'].unique()
    rp_edges = [(node, p) for p in providers]
    g.add_edges_from(rp_edges)
    
    # Add them to the graph
    for e in rp_edges:
        try:
            types = g.edges[e]['type']
            if not 'request_provide' in types:
                g.edges[e]['type'] += ['request_provide']
        except KeyError:
            g.edges[e]['type'] = ['request_provide']
            

    # Find edges based on branch
    # --------------------------
    # Find the node's parent
    parent = '.'.join(node.split('.')[:-1])
    s_edges = []
    if parent: 
        s_edges += [(parent, node)]    
    g.add_edges_from(s_edges)
        
    # Add them to the graph
    for e in s_edges:
        try:
            types = g.edges[e]['type']
            if not 'structure' in types:
                g.edges[e]['type'] += ['structure']
        except KeyError:
            g.edges[e]['type'] = ['structure']  

In [ ]:
for node in node_dfs:
    add_edges(node, node_dfs[node], g)
    
for node in tech_dfs:
    for tech in tech_dfs[node]:
        add_edges(node, tech_dfs[node][tech], g)

In [ ]:
from networkx.drawing.nx_agraph import graphviz_layout
pos = graphviz_layout(g, prog='dot')
plt.figure(figsize=(20,12))
nx.draw(g,pos,with_labels=True, font_size=7, node_color='gainsboro', font_color='tomato')
plt.show()

# Find the Supply Sub-graph

In [ ]:
# Find the demand sub-graph
# d_nodes = [n for n, a in g.nodes(data=True) if a['type'] in ('demand', 'standard')]
d_nodes = []
for n, a in g.nodes(data=True):
    try:
        a['type']
        if a['type'] in ['supply', 'standard']:
            d_nodes.append(n)
    except KeyError:
        pass

g_demand = g.subgraph(d_nodes).copy()

In [ ]:
pos = graphviz_layout(g_demand, prog='dot')
plt.figure(figsize=(20,12))
nx.draw(g_demand, pos,with_labels=True, font_size=7, node_color='gainsboro', font_color='tomato')
plt.show()

In [ ]:
import pickle

In [ ]:
pickle_out = open("graph.pickle","wb")
pickle.dump(g, pickle_out)
pickle_out.close()

In [ ]:
g.nodes['Canada']